In [1]:
import numpy as np
from pymatgen.core import Structure
from mendeleev import element  # For fetching covalent radius

# Function to compute atom and bond vectors
def compute_atom_and_bond_vectors(poscar_file):
    delta = 0.3  # Parameter for bond decision (sum of covalent radii + delta)

    # Open and read the POSCAR file to determine coordinate system
    with open(poscar_file, 'r') as f:
        lines = f.readlines()
        # Check if 'Direct' or 'Cartesian' appears in the file
        if any('direct' in line.lower() for line in lines):
            coord_type = 'direct'
        elif any('cartesian' in line.lower() for line in lines):
            coord_type = 'cartesian'
        else:
            raise ValueError("Coordinate type not found in the POSCAR file.")

    # Parse the POSCAR file using pymatgen
    structure = Structure.from_file(poscar_file)
    
    num_atoms = len(structure)
    atomic_numbers = [site.specie.Z for site in structure.sites]
    positions = np.array([site.frac_coords if coord_type == 'direct' else site.coords for site in structure.sites])
    lattice = structure.lattice.matrix
    if coord_type == 'direct':
        positions = np.dot(positions, lattice)  # Convert fractional to Cartesian if needed

    # Initialize the atom vector and bond vector
    atom_vector = np.zeros(118, dtype=int)
    bond_vector = np.zeros(118 * 119 // 2, dtype=int)  # Triangular matrix, 1-1, 1-2, ..., 118-118

    # Fill the atom vector
    for atomic_number in atomic_numbers:
        atom_vector[atomic_number - 1] += 1  # Atomic numbers start at 1, array indices at 0

    # Get covalent radii for bonding check
    covalent_radii = [element(site.specie.symbol).covalent_radius / 100 for site in structure.sites]

    # Fill the bond vector
    def bond_index(z1, z2):
        # Convert z1 and z2 to a unique index in the bond_vector for z1-z2 pairs
        return (z1 - 1) * 118 + (z2 - 1) - (z1 * (z1 - 1)) // 2

    # Loop over each pair of atoms and neighboring cells
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            for k in [-1, 0, 1]:
                for l in [-1, 0, 1]:
                    for m in [-1, 0, 1]:
                        translation = k * lattice[0] + l * lattice[1] + m * lattice[2]
                        distance_vector = positions[i] - (positions[j] + translation)
                        distance = np.linalg.norm(distance_vector)

                        sum_covalent_radii = covalent_radii[i] + covalent_radii[j] + delta
                        if distance <= sum_covalent_radii:
                            z1, z2 = sorted((atomic_numbers[i], atomic_numbers[j]))
                            bond_vector[bond_index(z1, z2)] += 1

    return atom_vector, bond_vector

# Example usage:
poscar_file = "POSCAR_Bilayers/As2+As2_4ce37585d2a3c90d+0001.vasp"
atom_vector, bond_vector = compute_atom_and_bond_vectors(poscar_file)
print("Atom Vector:", atom_vector)
##index rule for the bond n-m
#n=34
#m=82
#index=m - (n-1) + (n-1)*118 - (n*(n-1))//2
print("Bond Vector:", bond_vector)


C:\Users\lorenzo.xavier\AppData\Roaming\Python\Python39\site-packages\pymatgen\core\structure.py:3083: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:


Atom Vector: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
Bond Vector: [0 0 0 ... 0 0 0]


In [3]:
# Print non-zero elements of the bond_vector with their indices
non_zero_bonds = [(i, bond_vector[i]) for i in range(len(bond_vector)) if bond_vector[i] > 0]
print("Non-zero Bond Vector Elements:")

for index, count in non_zero_bonds:
    # Calculate Z1 and Z2 based on the flattened bond index
    z1 = 1
    cumulative_count = 0
    while index >= cumulative_count + (118 - z1 + 1):
        cumulative_count += (118 - z1 + 1)
        z1 += 1
    z2 = z1 + (index - cumulative_count) 

    print(f"Bonds between atomic numbers {z1}-{z2}: {count}")


Non-zero Bond Vector Elements:
Bonds between atomic numbers 33-33: 6


In [4]:
np.sum(bond_vector)

6

In [5]:
atom_vector

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
print(bond_vector)

[0 0 0 ... 0 0 0]
